In [ ]:
%matplotlib inline
from stack_nlp import *
from jupyter_mplsettings import *
matplotlib.rc('font', **font)

In [ ]:
cfg = local_import("./laptop.py")

In [ ]:
conn = sqlite3.connect(cfg.posts_path)

In [ ]:
from gensim.parsing import preprocessing

i = 0
for post in YieldDBPosts(conn):
    print post[0]
    print preprocessing.preprocess_string(post[0])
    i += 1
    if i == 3:
        break